In [10]:
import os
import pandas as pd
import altair as alt

In [11]:
def get_faker(filepath): #Function that takes filepath of one of the datasets and gets the required stats for Faker's games
    fp = os.path.join(filepath)
    league_raw = pd.read_csv(fp, low_memory=False)    
    league = league_raw[league_raw['position'] != 'team']
    faker = league[league['playername'] == 'Faker'].get(['year', 'patch', 'playername', 'teamname', 'champion', 'gamelength', 'earnedgold'])
    faker['gamelength'] = (faker['gamelength'] / 60).round(0)
    return faker

## Requirement

#### Make sure you download the datasets from [Google Drive](https://drive.google.com/drive/u/1/folders/1gLSw0RLjBbtaNy0dgnGQDAZOHIgCe-HH) (2020, 2021, 2022, 2023)

#### Put them in "data_processing" folder and rename them

In [12]:
faker_2023 = get_faker('lol_2023_data.csv')
faker_2022 = get_faker('lol_2022_data.csv')
faker_2021 = get_faker('lol_2021_data.csv')
faker_2020 = get_faker('lol_2020_data.csv')

In [13]:
faker_all = pd.concat([faker_2023, faker_2022, faker_2021, faker_2020])

In [14]:
faker_filtered = (faker_all.groupby('gamelength').filter(lambda x: len(x) >= 10) #Take only game lengths which were played 10 or more times
                  .groupby('champion').filter(lambda x: len(x) >= 7)) #Take only champions which were played 7 or more times

In [15]:
faker_grouped = faker_filtered.groupby(['gamelength', 'champion'])['earnedgold'].mean().reset_index() #Get mean earned gold for each champion and game length
faker_grouped = faker_grouped.sort_values(['champion', 'gamelength']) #must sort or graph will be weird
faker_grouped

,gamelength,champion,earnedgold
18,26.0,Ahri,6776.333333
31,27.0,Ahri,8370.000000
41,28.0,Ahri,7418.500000
64,30.0,Ahri,7053.500000
79,31.0,Ahri,9043.666667
...,...,...,...
104,32.0,Zoe,7006.750000
114,33.0,Zoe,9063.500000
125,34.0,Zoe,7667.000000
133,35.0,Zoe,8300.000000


In [16]:
faker_total = faker_grouped.groupby('gamelength')['earnedgold'].mean().reset_index() #Get overall earned gold per gamelength
faker_total = faker_total.sort_values('gamelength') #must sort or graph will be weird
faker_total

,gamelength,earnedgold
0,24.0,7226.285714
1,25.0,7223.045455
2,26.0,7115.153846
3,27.0,7644.090000
4,28.0,8123.357143
5,29.0,7989.796296
6,30.0,8408.488889
7,31.0,8129.900794
8,32.0,8168.118056
9,33.0,8113.620000


## Export to json files

In [17]:
faker_grouped.to_json('../../static/faker_grouped.json', orient='records')
faker_total.to_json('../../static/faker_total.json', orient='records')

In [18]:
champions = alt.Chart(faker_grouped, title = 'Earned Gold over Game Length, Per Champion').mark_line().encode(
    x = alt.X('gamelength').title('Game Length'),
    y = alt.Y('earnedgold').title('Earned Gold'),
    color = alt.Color('champion').title('Champions')
)
total = alt.Chart(faker_total, title = 'Earned Gold over Game Length Overall').mark_line().encode(
    x = alt.X('gamelength').title('Game Length'),
    y = alt.Y('earnedgold').title('Earned Gold'),
)

In [19]:
champions

alt.Chart(...)

In [ ]:
total